In [1]:
# importing and path definition
import math
import sys
import faiss
import h5py
import csv
from functools import wraps
from time import time
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.manifold import TSNE
import umap

In [2]:
HERE = Path(_dh[-1])
HERE

PosixPath('/media/arma/DATA/Proje_3501/LIBRARY/CommerciallyAvailable/SELECTED')

In [3]:
df = pd.read_csv('H20_selected.csv',  index_col=False, )
#df.drop(['Unnamed: 0'],axis=1, inplace=True)
#df = pd.read_csv('H10P0_all_filtered.csv', )
#df.columns=['a', 'b', 'remain', ]

#df.sort_values(by = 'ZINCID', inplace=True)
df

,SMILES,ZINCID,all_SMILES,all_ZINCID,all_CatalogName,all_Supplier
0,CC#CCC(NC(=O)CSCC(=O)OC)C(=O)OC,ZINCj5000002736T,CC#CC[C@@H](NC(=O)CSCC(=O)OC)C(=O)OC,ZINCj5000002736T,s_22__13586360__8749960,s
1,CC#CCC1NC(=O)N(CCS(=O)(=O)NCC)C1=O,ZINCj500000273aK,CC#CC[C@@H]1NC(=O)N(CCS(=O)(=O)NCC)C1=O,ZINCj500000273aK,s_272104__9342054__13586368,s
2,CC(=O)N1CCCC1C(=O)N=c1[nH][nH]cc1C(N)=O,ZINCj500000275NB,CC(=O)N1CCC[C@@H]1C(=O)N=c1[nH][nH]cc1C(N)=O,ZINCj500000275NB,s_22__13968744__12398288,sv
3,CC(=O)N1CCC(OC(=O)C2C3CS(=O)(=O)CC32)C1,ZINCj5000002779T,CC(=O)N1CC[C@@H](OC(=O)C2[C@@H]3CS(=O)(=O)C[C@...,ZINCj5000002779T,s_276436__14083982__16734506,s
4,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1,ZINCj500000278nY,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1,ZINCj500000278nY,s_22__3417378__1183354,s
...,...,...,...,...,...,...
481345,Cc1sc2ccccc2c1CNC1CCNC(C)C1,ZINCjE00000e7710,Cc1sc2ccccc2c1CN[C@@H]1CCN[C@H](C)C1,ZINCjE00000e7710,s_271302__22840822__25563868,sf
481346,CN(CC1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl,ZINCjE00000eiMfy,CN(C[C@@H]1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl,ZINCjE00000eiMfy,s_527__8963522__23091736,sf
481347,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O,ZINCjE000003Vqni,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O,ZINCjE000003Vqni,s_527__6582774__8794394,sv
481348,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1,ZINCjE000003YCQH,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1,ZINCjE000003YCQH,WSWJRZDITRUCID-UHFFFAOYSA-N,mcule


In [4]:
h5f = h5py.File("SPACE/H20_selected.h5", 'r')
h5f.keys()

<KeysViewHDF5 ['fp_list', 'name_list', 'smiles_list']>

In [5]:
x = h5f['fp_list'][:]
smiles_list = h5f['smiles_list'][:]
name_list = h5f['name_list'][:]
#h5f.close()

In [6]:
smiles_ls = []
for smiles in smiles_list:
    a = smiles[0].decode('utf-8')
    smiles_ls.append(a)

In [7]:
names_ls = []
for names in name_list:
    a = names[0].decode('utf-8')
    names_ls.append(a)

In [8]:
x.shape

(481350, 1024)

In [9]:
fp_df = pd.DataFrame.from_dict(zip(smiles_ls, names_ls, x))
fp_df.columns=['SMILES', 'ZINCID', 'FP', ]
fp_df

,SMILES,ZINCID,FP
0,CC#CCC(NC(=O)CSCC(=O)OC)C(=O)OC,ZINCj5000002736T,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,CC#CCC1NC(=O)N(CCS(=O)(=O)NCC)C1=O,ZINCj500000273aK,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,CC(=O)N1CCCC1C(=O)N=c1[nH][nH]cc1C(N)=O,ZINCj500000275NB,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,CC(=O)N1CCC(OC(=O)C2C3CS(=O)(=O)CC32)C1,ZINCj5000002779T,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1,ZINCj500000278nY,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...,...
481345,Cc1sc2ccccc2c1CNC1CCNC(C)C1,ZINCjE00000e7710,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
481346,CN(CC1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl,ZINCjE00000eiMfy,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
481347,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O,ZINCjE000003Vqni,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
481348,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1,ZINCjE000003YCQH,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [10]:
df_fpAll = pd.merge(df, fp_df, on=['ZINCID', 'SMILES'], how='inner')
#df_fpAll = df_fpAll[['molecule_chembl_id','label', 'fp_MACCS', 'fp_Morgan3', 'fp_MAP4']]
df_fpAll

,SMILES,ZINCID,all_SMILES,all_ZINCID,all_CatalogName,all_Supplier,FP
0,CC#CCC(NC(=O)CSCC(=O)OC)C(=O)OC,ZINCj5000002736T,CC#CC[C@@H](NC(=O)CSCC(=O)OC)C(=O)OC,ZINCj5000002736T,s_22__13586360__8749960,s,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,CC#CCC1NC(=O)N(CCS(=O)(=O)NCC)C1=O,ZINCj500000273aK,CC#CC[C@@H]1NC(=O)N(CCS(=O)(=O)NCC)C1=O,ZINCj500000273aK,s_272104__9342054__13586368,s,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,CC(=O)N1CCCC1C(=O)N=c1[nH][nH]cc1C(N)=O,ZINCj500000275NB,CC(=O)N1CCC[C@@H]1C(=O)N=c1[nH][nH]cc1C(N)=O,ZINCj500000275NB,s_22__13968744__12398288,sv,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
3,CC(=O)N1CCC(OC(=O)C2C3CS(=O)(=O)CC32)C1,ZINCj5000002779T,CC(=O)N1CC[C@@H](OC(=O)C2[C@@H]3CS(=O)(=O)C[C@...,ZINCj5000002779T,s_276436__14083982__16734506,s,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1,ZINCj500000278nY,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1,ZINCj500000278nY,s_22__3417378__1183354,s,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, ..."
...,...,...,...,...,...,...,...
481345,Cc1sc2ccccc2c1CNC1CCNC(C)C1,ZINCjE00000e7710,Cc1sc2ccccc2c1CN[C@@H]1CCN[C@H](C)C1,ZINCjE00000e7710,s_271302__22840822__25563868,sf,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
481346,CN(CC1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl,ZINCjE00000eiMfy,CN(C[C@@H]1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl,ZINCjE00000eiMfy,s_527__8963522__23091736,sf,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
481347,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O,ZINCjE000003Vqni,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O,ZINCjE000003Vqni,s_527__6582774__8794394,sv,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
481348,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1,ZINCjE000003YCQH,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1,ZINCjE000003YCQH,WSWJRZDITRUCID-UHFFFAOYSA-N,mcule,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [11]:
fp_Morgan_1024 = list((df_fpAll['FP'].values))
len(fp_Morgan_1024)

481350

In [12]:
#pca_Morgan = PCA(n_components=2)
#crds_MACCS = pca_Morgan.fit_transform(fp_Morgan_1024) 
#var = np.sum(pca_Morgan.explained_variance_ratio_)
#var

In [13]:
#def evaluate_components(fp_list):
#    res = []
#    for n_comp in tqdm(range(2,25)):
#        pca = PCA(n_components=n_comp)
#        crds = pca.fit_transform(fp_list) 
#        var = np.sum(pca.explained_variance_ratio_)
#        res.append([n_comp,var])
#    return res

In [14]:
#res_Morgan_1024 = evaluate_components(fp_Morgan_1024)

In [15]:
#res_df_Morgan_1024 = pd.DataFrame(res_Morgan_1024,columns=["Components","Variance"])
#res_df_Morgan_1024['FP'] = 'Morgan 256 bits'
#res_df_Morgan_1024

In [16]:
#ax = sns.lineplot(x="Components",y="Variance",hue="FP",data=res_df_Morgan_1024,linewidth=2)

In [17]:
#pca_Morgan_1024 = PCA(n_components=100)
#crds_Morgan_1024 = pca_Morgan_1024.fit_transform(fp_Morgan_1024) 
#crds_Morgan_1024_embedded = TSNE(n_components=2, perplexity=40, n_iter=5000, random_state=10, init='pca').fit_transform(crds_Morgan_1024)
#tsne_Morgan_1024_df = pd.DataFrame(crds_Morgan_1024_embedded,columns=["X","Y"])
#tsne_Morgan_1024_df['label'] = df_fpAll['label']

In [18]:
#tsne_Morgan_1024_df['Name'] = df_fpAll['ZINCID']
#tsne_Morgan_1024_df['SMILES'] = df_fpAll['SMILES']
#tsne_Morgan_1024_df.to_csv('tsne_Morgan256.csv')

In [19]:
#tsne_Morgan_1024_df

In [20]:
#fig = plt.figure(figsize=(12, 10))
#ax = sns.scatterplot(data=tsne_Morgan_1024_df,x="X",y="Y",)
#ax.set_xlabel('t-SNE1', fontsize=18)
#ax.set_ylabel('t-SNE2', fontsize=18)
#ax.legend(fontsize=14, facecolor='w', loc=4)
#plt.tight_layout()

#fig.legend(loc='upper right', bbox_to_anchor=(.75, 0.98))
#plt.show()

In [21]:
#tsne_Morgan_1024_df.to_csv('../SELECTED/H25_tsne.csv', index=False)

In [22]:
clusterable_embedding = umap.UMAP(
    n_neighbors=25,
    min_dist=0.0,
    n_components=2,
    #random_state=42,
).fit_transform(fp_Morgan_1024)

In [23]:
umap_Morgan_1024_df = pd.DataFrame(clusterable_embedding[:,:2],columns=["X","Y"])
umap_Morgan_1024_df['Name'] = df_fpAll['ZINCID']
umap_Morgan_1024_df['SMILES'] = df_fpAll['SMILES']
umap_Morgan_1024_df

,X,Y,Name,SMILES
0,11.704470,-2.818953,ZINCj5000002736T,CC#CCC(NC(=O)CSCC(=O)OC)C(=O)OC
1,12.535139,3.060397,ZINCj500000273aK,CC#CCC1NC(=O)N(CCS(=O)(=O)NCC)C1=O
2,11.081284,0.508415,ZINCj500000275NB,CC(=O)N1CCCC1C(=O)N=c1[nH][nH]cc1C(N)=O
3,13.214011,0.572323,ZINCj5000002779T,CC(=O)N1CCC(OC(=O)C2C3CS(=O)(=O)CC32)C1
4,11.656877,-2.537296,ZINCj500000278nY,CC(=O)NCC(=O)NCC(=O)NC1(C#N)CCCC1
...,...,...,...,...
481345,7.786683,-0.250963,ZINCjE00000e7710,Cc1sc2ccccc2c1CNC1CCNC(C)C1
481346,11.757635,0.293895,ZINCjE00000eiMfy,CN(CC1CCCO1)C(=O)c1c(F)cc(Br)cc1Cl
481347,10.633849,-0.284853,ZINCjE000003Vqni,CCCCN(CCCC)C(=O)c1nccc(Cl)c1O
481348,9.493270,1.002663,ZINCjE000003YCQH,O=C(N=c1[nH][nH]cc1Br)c1cccc(C(F)(F)F)c1


In [24]:
fig = plt.figure(figsize=(12, 10))
ax = sns.scatterplot(data=umap_Morgan_1024_df,x="X",y="Y",)
ax.set_xlabel('umap1', fontsize=18)
ax.set_ylabel('umap2', fontsize=18)
ax.legend(fontsize=14, facecolor='w', loc=4)
plt.tight_layout()

#fig.legend(loc='upper right', bbox_to_anchor=(.75, 0.98))
plt.show()

/tmp/ipykernel_84310/3040977561.py:5: UserWarning: No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.
  ax.legend(fontsize=14, facecolor='w', loc=4)


In [25]:
umap_Morgan_1024_df.to_csv("SPACE/H20_umap.csv", index=False)